In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pymysql

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
conn=pymysql.connect(host="localhost",user="root",passwd="liugang666",db="book_crossing",charset="utf8")

In [3]:
query_bookRatings_sql = "select * from book_ratings"

In [4]:
ratings_df = pd.read_sql(query_bookRatings_sql,con=conn)

In [5]:
ratings_df.tail()

,User-ID,ISBN,Book-Rating
825833,278854,0425163393,7
825834,278854,0515087122,0
825835,278854,0553275739,6
825836,278854,0553578596,0
825837,278854,0553579606,8


In [6]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,2,0195153448,0
1,7,034542252,0
2,8,0002005018,5
3,8,0060973129,0
4,8,0374157065,0


In [7]:
query_books_sql = "select `ISBN`,`Book-Title` from books"

In [8]:
books_df = pd.read_sql(query_books_sql,con=conn)

In [9]:
books_df.head()

,ISBN,Book-Title
0,0000913154,The Way Things Work: An Illustrated Encycloped...
1,0001010565,Mog's Christmas
2,0001046438,Liar
3,0001046713,Twopence to Cross the Mersey
4,000104687X,"T.S. Eliot Reading ""The Wasteland"" and Other P..."


In [10]:
books_df.tail()

,ISBN,Book-Title
271374,B000234N76,Falling Angels
271375,B000234NC6,It Must've Been Something I Ate: The Return of...
271376,B00029DGGO,"Good Wife Strikes Back, The"
271377,B0002JV9PY,The Blockade Runners
271378,B0002K6K8O,The Underground City


In [11]:
conn.close()

In [12]:
#将ISBN替换为行号以节省空间方便运算

In [13]:
books_df['booksRow'] = books_df.index

In [14]:
books_df.head()

,ISBN,Book-Title,booksRow
0,0000913154,The Way Things Work: An Illustrated Encycloped...,0
1,0001010565,Mog's Christmas,1
2,0001046438,Liar,2
3,0001046713,Twopence to Cross the Mersey,3
4,000104687X,"T.S. Eliot Reading ""The Wasteland"" and Other P...",4


In [15]:
ratings_df = pd.merge(ratings_df,books_df,on="ISBN")

In [16]:
ratings_df.head()

,User-ID,ISBN,Book-Rating,Book-Title,booksRow
0,2,0195153448,0,Classical Mythology,25030
1,8,0002005018,5,Clara Callan,73
2,11400,0002005018,0,Clara Callan,73
3,11676,0002005018,8,Clara Callan,73
4,41385,0002005018,0,Clara Callan,73


In [17]:
ratings_df = ratings_df[['User-ID','booksRow','Book-Rating']]

In [18]:
ratings_df.head()

,User-ID,booksRow,Book-Rating
0,2,25030,0
1,8,73,5
2,11400,73,0
3,11676,73,8
4,41385,73,0


In [19]:
ratings_df.to_csv('ratingsProcessed.csv',index=False,header=True,encoding='utf-8')

In [20]:
books_df = books_df[['booksRow','Book-Title']]

In [21]:
books_df.to_csv('booksProcessed.csv',index=False,header=True,encoding='utf-8')

In [22]:
books_df.head()

,booksRow,Book-Title
0,0,The Way Things Work: An Illustrated Encycloped...
1,1,Mog's Christmas
2,2,Liar
3,3,Twopence to Cross the Mersey
4,4,"T.S. Eliot Reading ""The Wasteland"" and Other P..."


In [23]:
userNo = ratings_df['User-ID'].max()+1

In [24]:
bookNo = ratings_df['booksRow'].max()+1

In [25]:
userNo

278855

In [26]:
bookNo

271379

In [27]:
from scipy.sparse import dok_matrix

In [28]:
rating = dok_matrix((userNo,bookNo), dtype=np.float32)

In [29]:
rating

<278855x271379 sparse matrix of type '<class 'numpy.float32'>'
	with 0 stored elements in Dictionary Of Keys format>

In [30]:
for index,row in ratings_df.iterrows():
    rating[int(row['User-ID']),int(row['booksRow'])] = row['Book-Rating']

In [31]:
np.shape(rating)

(278855, 271379)

In [34]:
record = rating>0

MemoryError: 

In [32]:
for index,row in ratings_df.iterrows():
    rating[(int(row['User-ID']),int(row['booksRow']))] = row['Book-Rating']
    record[(int(row['User-ID']),int(row['booksRow']))] = 1
    #flag +=1
    #ratings_df_length -=1
    #print('Processed %d, %d left' % (flag,ratings_df_length))

#构建模型

In [48]:
num_f = 20

In [49]:
U_parameters = tf.Variable(tf.random_normal([userNo,num_f],stddev=0.35))

In [55]:
print(U_parameters)

<tf.Variable 'Variable:0' shape=(278855, 20) dtype=float32_ref>


In [72]:
B_parameters = tf.Variable(tf.random_normal([bookNo,num_f],stddev=0.35))

In [75]:
loss = tf.reduce_sum((tf.matmul(U_parameters,B_parameters,transpose_b=True)-rating)**2)+tf.reduce_sum(U_parameters**2)+tf.reduce_sum(B_parameters**2)

TypeError: unsupported operand type(s) for -: 'Tensor' and 'dok_matrix'